# DataSet

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
cat_image_path = "../data/cifar10_images/cat"
dog_image_path = "../data/cifar10_images/dog"

In [3]:
def load_image_from_path(path):
    images = []
    labels = []
    label = path.split('/')[-1]
    for file_name in os.listdir(path):
        image_path = os.path.join(path, file_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (32, 32))
        image = image.reshape(-1)
        image = image / 255.0
        images.append(image)

        if label == 'cat':
            labels.append(0)
        elif label == 'dog':
            labels.append(1)
        else:
            raise ValueError(f"Unexpected label: {label}. Expected 'cat' or 'dog'.")

    return images, labels

In [4]:
cat_images, cat_labels = load_image_from_path(cat_image_path)
dog_images, dog_labels = load_image_from_path(dog_image_path)

print(f"data shape: {cat_images[0].shape}")
print(f"label shape: {cat_labels[0]}")
print(f"number of cat data: {len(cat_labels)}")
print(f"number of dog data: {len(dog_labels)}")

data shape: (3072,)
label shape: 0
number of cat data: 500
number of dog data: 500


In [5]:
images = np.array(cat_images + dog_images)
labels = np.array(cat_labels + dog_labels)

print(f"number of all data: {len(labels)}")

number of all data: 1000


In [6]:
train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"number of train data: {len(train_y)}")
print(f"number of test data: {len(test_y)}")
print(f"check data set randomly: {test_y[:10]}")

number of train data: 800
number of test data: 200
check data set randomly: [1 1 1 1 0 1 1 1 1 0]


In [7]:
train_y = train_y.reshape((800, 1))
test_y = test_y.reshape((200, 1))

# HyperParameter

In [8]:
num_features = 3072
num_samples = 800
learning_rate = 0.1
num_iterations = 1000
hidden_layer_size = 128

# Model

In [9]:
W1 = np.random.randn(num_features, hidden_layer_size) * 0.01
b1 = np.zeros((1, hidden_layer_size))
W2 = np.random.randn(hidden_layer_size, 1) * 0.01
b2 = np.zeros((1, 1))

# Loss

In [10]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Numerical stability
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    return loss

# Activation function

In [11]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [12]:
def relu(z):
    return np.maximum(0, z)

# Train

In [13]:
for i in range(num_iterations):
    Z1 = np.dot(train_x, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    y_hat = sigmoid(Z2)

    loss = binary_cross_entropy(train_y, y_hat)

    dZ2 = y_hat - train_y
    dW2 = np.dot(A1.T, dZ2) / num_samples
    db2 = np.sum(dZ2, axis=0, keepdims=True) / num_samples
    
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * (Z1 > 0)
    dW1 = np.dot(train_x.T, dZ1) / num_samples
    db1 = np.sum(dZ1, axis=0, keepdims=True) / num_samples

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {loss}")

print(f"Final Loss: {loss}")

Iteration 0, Loss: 0.6933912525141142
Iteration 100, Loss: 0.6595873309225564
Iteration 200, Loss: 0.7019586885822343
Iteration 300, Loss: 0.5638684578754173
Iteration 400, Loss: 0.5523438212703811
Iteration 500, Loss: 0.6274251174498421
Iteration 600, Loss: 0.49719117831275866
Iteration 700, Loss: 0.6466506685038893
Iteration 800, Loss: 0.3880684014564935
Iteration 900, Loss: 0.5639808901190831
Final Loss: 0.5535857684869286


# Predict

In [14]:
Z1_test = np.dot(test_x, W1) + b1
A1_test = relu(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
test_y_hat = sigmoid(Z2_test)

test_y_pred = (test_y_hat > 0.5).astype(int)
accuracy = np.mean(test_y_pred == test_y)

print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

Test Set Accuracy: 59.50%
